## ERRANT for Mandarin GEC

### References
#### https://github.com/getao/errant2. 
#### https://www.aclweb.org/anthology/P17-1074/3. 
#### https://www.repository.cam.ac.uk/handle/1810/293719
#### https://github.com/ckiplab/ckiptagger (install and download model files)

# Using CkipTagger
中研院斷詞工具

In [1]:
# Visit https://github.com/ckiplab/ckiptagger to install and download model files
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import numpy as np
import nltk

ws = WS("/home/ginger/practice_7/ckiptagger/data")
pos = POS("/home/ginger/practice_7/ckiptagger/data")

In [2]:
import re
def before_edit(text):
    return re.sub('(\[\-|\-\]|\{\+.*?\+\})', '', text)
    
def after_edit(text):
    return re.sub('\[\-.*?\-\]|\{\+|\+\}', '', text)

text = '[-這-]也[-間-][-接-]{+直+}{+接+}{+透露+}{+出+}[-突-][-顯-][-出-]鴻海'

text_before = ' '.join(ws(before_edit(text).splitlines())[0])
print (text_before)

text_after = ' '.join(ws(after_edit(text).splitlines())[0])
print (text_after)

這 也 間接 突顯出 鴻海
也 直接 透露出 鴻海


# [Step 1] 加入詞性參考 by ckip
input
``` Python
text = '這也間接突顯出鴻海'
text = '也直接透露出鴻海'

```
return
``` Python
[('這', 'Nep'), ('也', 'D'), ('間接', 'D'), ('突顯出', 'VJ'), ('鴻海', 'Nb')]
[('也', 'D'), ('直接', 'VH'), ('透露出', 'VK'), ('鴻海', 'Nb')]
```

In [3]:
def seg_pos(text):
    ##### Your code here #####
    return [(x,y) for x,y in zip(ws([text])[0],pos(ws([text]))[0])]

print (seg_pos(before_edit(text)))
print (seg_pos(after_edit(text)))

[('這', 'Nep'), ('也', 'D'), ('間接', 'D'), ('突顯出', 'VJ'), ('鴻海', 'Nb')]
[('也', 'D'), ('直接', 'VH'), ('透露出', 'VK'), ('鴻海', 'Nb')]


In [4]:
sent_orig = seg_pos(before_edit(text))
sent_correct = seg_pos(after_edit(text)) 
sent_orig, sent_correct

([('這', 'Nep'), ('也', 'D'), ('間接', 'D'), ('突顯出', 'VJ'), ('鴻海', 'Nb')],
 [('也', 'D'), ('直接', 'VH'), ('透露出', 'VK'), ('鴻海', 'Nb')])

# [Strp 2] 計算 edit cost 返回cost_matrix, op_matrix
`Hint:[Strp 2]會呼叫[Strp 3]的方法，Cost可以自己調整`

```
M是Match
I是insert cost
D是delete cost
R是replace cost
I/D是insert和delete相加的cost
```
input
``` Python
orig = [('這', 'Nep'), ('也', 'D'), ('間接', 'D'), ('突顯出', 'VJ'), ('鴻海', 'Nb')]
cor = [('也', 'D'), ('直接', 'VH'), ('透露出', 'VK'), ('鴻海', 'Nb')]

```
return
``` Python
cost_matrix = 
 [[0, 2, 4, 5, 4],
  [4, 6, 8, 9, 8],
  [2, 0, 6, 7, 6],
  [3, 5, 5, 8, 7],
  [5, 7, 8, 5, 8],
  [4, 6, 8, 9, 0]]
    
op_matrix =
 [['O', 'I', 'I', 'I', 'I'],
  ['D', 'I/D', 'I/D', 'I/D', 'I/D'],
  ['D', 'M', 'I/D', 'I/D', 'I/D'],
  ['D', 'I/D', 'R', 'I/D', 'I/D'],
  ['D', 'I/D', 'R', 'R', 'R'],
  ['D', 'I/D', 'I/D', 'I/D', 'M']]
```

In [5]:
def calculate_edit_cost(orig, cor):
    o_len = len(orig)
    c_len = len(cor)
    # Create the cost_matrix and the op_matrix
    cost_matrix = [[0 for j in range(c_len+1)] for i in range(o_len+1)]
    op_matrix = [["O" for j in range(c_len+1)] for i in range(o_len+1)]
    # Fill in the edges
    for i in range(1, o_len+1):
        cost_matrix[i][0] = nltk.edit_distance(orig[i-1][1] + orig[i-1][0], '')
        op_matrix[i][0] = "D"
    for j in range(1, c_len+1):
        cost_matrix[0][j] = nltk.edit_distance('', cor[j-1][1] + cor[j-1][0])
        op_matrix[0][j] = "I"

    # Loop through the cost_matrix
    for i in range(o_len):
        for j in range(c_len):
            # Matches
            if orig[i] == cor[j]:
                cost_matrix[i+1][j+1] = 0
                op_matrix[i+1][j+1] = "M"
            # Non-matches
            else:
                ##### Your code here #####
                sub_cost = get_sub_cost(orig[i], cor[j]) + abs(i-j)
                ID_cost = cost_matrix[i+1][0] + cost_matrix[0][j+1]
                if sub_cost < ID_cost:
                    op_matrix[i+1][j+1] = 'R'
                    cost_matrix[i+1][j+1] = sub_cost
                else:
                    op_matrix[i+1][j+1] = 'I/D'
                    cost_matrix[i+1][j+1] = ID_cost
                
    # Return the matrices
    return cost_matrix, op_matrix

# [Strp 3] 計算Replace的cost
嘗試找出可能可以計算cost的方式，例如檢查詞性、詞的長度或詞的差異等，最後將所有的cost相加回傳。

`Hint 要給多少Cost可以自己設定`

input
``` Python
orig = ('突顯出', 'VJ')
cor = ('透露出', 'VK')

```
return cost

In [6]:
def get_sub_cost(o, c):
    if o == c: return 0
    ##### Your code here #####
    cost = 0
    o = list(o)
    c = list(c)
    
    # word
    # compare length
    if len(o[0]) < len(c[0]):
        cost += 1
        o[0] += ' ' * (len(c[0])-len(o[0]))
    elif len(o[0]) > len(c[0]):
        cost += 1
        c[0] += ' ' * (len(o[0])-len(c[0]))
    # compare difference
    for i in range(len(o[0])):
        if c[0][i] != o[0][i]:
            cost += 1
    
    # part of speech
    if len(o[1]) < len(c[1]):
        o[1] += ' ' * (len(c[1])-len(o[1]))
    elif len(o[1]) > len(c[1]):
        c[1] += ' ' * (len(o[1])-len(c[1]))
    # compare difference
    for j in range(len(o[1])):
        if c[1][j] != o[1][j]:
            cost += 2
        
    return cost

In [7]:
calculate_edit_cost(sent_orig, sent_correct)

([[0, 2, 4, 5, 4],
  [4, 6, 8, 9, 8],
  [2, 0, 6, 7, 6],
  [3, 5, 6, 8, 7],
  [5, 7, 8, 5, 8],
  [4, 6, 8, 9, 0]],
 [['O', 'I', 'I', 'I', 'I'],
  ['D', 'I/D', 'I/D', 'I/D', 'I/D'],
  ['D', 'M', 'I/D', 'I/D', 'I/D'],
  ['D', 'I/D', 'R', 'I/D', 'I/D'],
  ['D', 'I/D', 'R', 'R', 'R'],
  ['D', 'I/D', 'I/D', 'I/D', 'M']])

# [Step 4] format cost_matrix and op_matrix

return
``` Python
[(0, 'D', '這'), (2, 'R', '間接', '直接'), (3, 'R', '突顯出', '透露出')]
```

In [8]:
def format_align(cost_matrix, op_matrix, orig, cor):
    ##### Your code here #####
    output = []
    
    for i in range(len(op_matrix)):
        
        # if no replacement and no match, then delete
        if op_matrix[i][0] == 'D':
            if ('R' not in op_matrix[i]) and ('M' not in op_matrix[i]):
                output.append((i-1, 'D', orig[i-1]))
        
        # if replace, get word which has min_cost as replacement
        min_cost = 99
        if 'R' in op_matrix[i]:
            for j in range(len(op_matrix[i])):
                if op_matrix[i][j] == 'R':
                    if cost_matrix[i][j] < min_cost:
                        min_cost = cost_matrix[i][j]
                        pos, new_pos = i, j
            output.append((i-1, 'R', orig[pos-1], cor[new_pos-1]))
        
    return output

In [9]:
s = ws(before_edit(text).splitlines())[0]
t = ws(after_edit(text).splitlines())[0]
print(s)
print(t)
cost_matrix, op_matrix = calculate_edit_cost(sent_orig, sent_correct)
format_align(cost_matrix, op_matrix, s, t)

['這', '也', '間接', '突顯出', '鴻海']
['也', '直接', '透露出', '鴻海']


[(0, 'D', '這'), (2, 'R', '間接', '直接'), (3, 'R', '突顯出', '透露出')]